In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
#from PIL import Image
import os
from keras import layers

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

#Data is downloaded into the container via shellscript. We believe this is more efficient
traindirectory="./archive/FaceMaskDataset/Train"
testdirectory="./archive/FaceMaskDataset/Test"
validationdirectory="./archive/FaceMaskDataset/Validation"
image_size=224
TrainData=keras.utils.image_dataset_from_directory(traindirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
TestData=keras.utils.image_dataset_from_directory(testdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
ValidationData=keras.utils.image_dataset_from_directory(validationdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))

img_augmentation = keras.models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(image_size, image_size),
  layers.Rescaling(1./255)
])


def cast_img(image, label):
    return tf.cast(image, tf.float16), label

def build_model(num_classes, IMG_SIZE):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    #x = img_augmentation(inputs) #image augmentation within the model. Should this be good practice? Or do we do it inside the map.
    x=inputs
    model = keras.applications.EfficientNetB0(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", dtype='float32', name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False), metrics=["accuracy"]
    )
    return model



from datetime import datetime
strategy = tf.distribute.MirroredStrategy()
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF


strategy = tf.distribute.MirroredStrategy()
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
#options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF


#unbatching as keras.utils.image_dataset_from_directory comes with a default batch
TrainData=TrainData.unbatch().with_options(options)
TestData=TestData.unbatch().with_options(options)
ValidationData=ValidationData.unbatch().with_options(options)

#TrainData=TrainData.map(lambda x, y: (img_augmentation(x), y),num_parallel_calls = tf.data.AUTOTUNE)
TrainData=TrainData.prefetch(tf.data.AUTOTUNE).batch(64*strategy.num_replicas_in_sync)
TestData=TestData.prefetch(tf.data.AUTOTUNE).batch(64*strategy.num_replicas_in_sync)
ValidationData=ValidationData.prefetch(tf.data.AUTOTUNE).batch(64*strategy.num_replicas_in_sync)

logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

early_stop= tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2
)

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logs,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')
with strategy.scope():
    model=build_model(2, 224)
model.fit(TrainData,
        epochs=1,
          validation_data=TestData
         , callbacks=[tboard_callback,early_stop])

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce GTX 1650 with Max-Q Design, compute capability 7.5
Found 10000 files belonging to 2 classes.
Found 992 files belonging to 2 classes.
Found 800 files belonging to 2 classes.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
157/157 [==============================] - 148s 591ms/step - loss: nan - accuracy: 0.9874 - val_loss: nan - val_accuracy: 0.5131


In [3]:
#EFFICIENT NET
model.evaluate(TrainData)

157/157 [==============================] - 47s 298ms/step - loss: nan - accuracy: 0.5000


[nan, 0.5]

In [4]:
#NORMAL MODEL

from tensorflow.keras import datasets, layers, models
with strategy.scope():
    model2 = models.Sequential()
    model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model2.add(layers.MaxPooling2D((2, 2)))
    model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model2.add(layers.MaxPooling2D((2, 2)))
    model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model2.add(layers.Flatten())
    model2.add(layers.Dense(64, activation='relu'))
    model2.add(layers.Dense(2, activation="softmax", dtype='float32'))
    model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model2.fit(TrainData,
        epochs=5,
          validation_data=TestData
         , callbacks=[tboard_callback,early_stop])

Epoch 1/5


C:\Users\seang\anaconda3\envs\tf_gpu\lib\site-packages\keras\backend.py:4906: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


157/157 [==============================] - 86s 372ms/step - loss: 66.1441 - accuracy: 0.8679 - val_loss: 0.1867 - val_accuracy: 0.9587
Epoch 2/5
157/157 [==============================] - 49s 309ms/step - loss: 0.1294 - accuracy: 0.9481 - val_loss: 0.1122 - val_accuracy: 0.9567
Epoch 3/5
157/157 [==============================] - 49s 308ms/step - loss: 0.0691 - accuracy: 0.9710 - val_loss: 0.1028 - val_accuracy: 0.9728
Epoch 4/5
157/157 [==============================] - 50s 315ms/step - loss: 0.0712 - accuracy: 0.9774 - val_loss: 0.1046 - val_accuracy: 0.9657
Epoch 5/5
157/157 [==============================] - 49s 308ms/step - loss: 0.0615 - accuracy: 0.9803 - val_loss: 0.1330 - val_accuracy: 0.9597
